<h1>Análises com dados do Strava Metro</h1>
<h2>Carregando arquivos .csv e importnado bibliotecas</h2>

In [1]:
import pandas as pd
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import numpy as np
from branca.colormap import linear
from ipywidgets import Dropdown, Output, VBox
from IPython.display import display, clear_output
#from ipywidgets import interact

month = ['jan', 'fev', 'mar', 'abr', 
          'mai', 'jun', 'jul', 'ago', 
          'set', 'out', 'nov', 'dez']
monthNum = ['12', '01', '02', '03',
            '04', '05', '06', '07',
            '08', '09', '10', '11']
year = 23
n = 17
data = [0 for _ in range(n + 1)]
perHour = [0 for _ in range(24)]


In [ ]:
for i in range(1, n):
    yearUpdate = year + ((i - 1) // 12)
    df = pd.read_csv(f'csvs/20{yearUpdate}-{monthNum[i % 12]}.csv')
    df['time'] = df['hour'].str.split('T').str[1]
    grouped = df.groupby('time')['total_trip_count'].sum()
    for time, total_trip_count in grouped.items():
        hour_index = int(time) 
        perHour[hour_index] += total_trip_count
        data[i] = df['total_trip_count'].sum()

print(data)

In [ ]:
total2= 0
total = 0

for i in range(24):
    total += perHour[i]

for i in range(13):
    total2 += 0

print(total)
print(total2)

<h2>Total de viagens por mês</h2>

In [ ]:
def total_de_viagens(data):
    plt.figure(figsize=(14, 6))

    x_positions = np.arange(1, n)
    x_labels = [f'{month[i % 12 - 1]}' + '/' + f'{year + (i // 12)}' for i in range(1, n)]
    for i in range(1, n):
        plt.bar(i, data[i] / 1000000, color = 'red')
        plt.text(i, data[i] / 1000000, f'{data[i] / 1000000:.2f}', ha='center', va='bottom')

    plt.xticks(x_positions, x_labels)
    plt.title('Viagens por mês')
    plt.xlabel('Mês/Ano')
    plt.ylabel('Quantidade de viagens no mês (em milhões)')
    plt.grid(True, axis='y', linestyle='--', alpha=0.7)
    plt.show()

total_de_viagens(data)

<h2>edges</h2>

In [ ]:
edges = gpd.read_file('edges/d71663f99c55ec3269ebde83b71ba106b038c992e6481967d9a95e80e79ba5f6-1716492978093.shp')
edges.crs = 'epsg:4326'

map_sp = folium.Map(location=[-23.5789, -46.6388], tiles="CartoDB Positron", zoom_start=11)

dropdown = Dropdown(
    options = [('Select Zone', None), ('Blue', 'blue'), ('Yellow', 'yellow'), ('Red', 'red')],
    value = None,
    description = 'color: ' 
)

map_output = Output()

def update_map(color):
    with map_output:
        clear_output(wait=True)
        if color is None:
            display(map_sp)
            return
        # #df2 = pd.read_csv(f'csvs/2023-01.csv')
        
        # grouped_df = df2.groupby('edge_uid')['total_trip_count'].sum().reset_index()
        # merged_data = pd.merge(grouped_df, edges, left_on='edge_uid', right_on='edgeUID')
    
        # min_trip_count = merged_data['total_trip_count'].min()
        # max_trip_count = 250
        
        # #colormap = linear.YlOrRd_09.scale(min_trip_count, max_trip_count)
    
        # for _, row in edges.iterrows():
        #     #color = colormap(row['total_trip_count'])

        map2 = folium.Map(location=[-23.5789, -46.6388], tiles="CartoDB Positron", zoom_start=11)
        folium.GeoJson(
            #row['geometry'],
            edges,
            style_function=lambda feature: {
                'color': color,
                'weight': 2,
                'opacity': 1,
            }
        ).add_to(map2)
    
        display(map2)

    #colormap.add_to(map_sp)

dropdown.observe(lambda change: update_map(change['new']), names='value')

display(VBox([dropdown, map_output]))

with map_output:
    display(map_sp)